### Importar librerias

Es necesario importar las librerias a utilizar, se usará spacy, pandas, 

In [0]:
# Ejecutar este fragmento si se ejecuta en colab
!pip install unidecode
!python -m spacy download es_core_news_sm

In [1]:
# Ejecutar este fragmento si se ejecuta en colab
# Debes autorizar a colab a acceder a tus archivos
from google.colab import drive
import os
drive.mount('/lab')

# En esta variable va la carpeta donde están los laboratorios en drive
carpeta_laboratorios = 

path_laboratorios = '/lab/My Drive/' + carpeta_laboratorios + '/'
print('Los archivos en tu carpeta de laboratoros son:')
os.listdir(path_laboratorios)

SyntaxError: invalid syntax (<ipython-input-1-2a534f3c8ddb>, line 8)

In [1]:
import nltk
import spacy
# from spacy import displacy
import pandas as pd
import re, string
import unidecode

#### Practicar Expresiones Regulares

Dada una lista de palabras, imprimir los elementos que contengan un determinado string
1. en cualquier lugar de la palabra
2. teriminen en ese string

Puede realizarlo usando la funcion match o la search

In [3]:
# ejemplo de busqueda de patron
# a) contienen era

lista = ['pera', 'espera','camino','marinerala', 'eramente']
for elem in lista:
    if re.match(r'.*era.*',elem):
        print(elem)


pera
espera
marinerala
eramente


In [4]:
# el search es como el match pero busca la ER en cualquier lugar de la cadena 
# sin necesidad de poner los símbolos de comienzo o fin poner el .*
# a) variante

lista = ['pera', 'espera','camino','marinerala','eramente']


In [5]:
# b) terminan en era y antes puede haber cualquier otro símbolo

lista = ['pera', 'espera','camino','marinerala', 'eramente']
for elem in lista:
    if re.match(r'.*(era)$',elem):
        print(elem)


pera
espera


#### Practicar sustituciones

Dada la oracion    'Si mañana vengo mañana voy a ver si consigo que mañana sea hoy' 

Sustituya las ocurrencias de la palabra mañana por la palabra hoy

In [6]:
#ejemplo de sustitución

oracion = 'Si mañana vengo mañana voy a ver si consigo que mañana sea hoy'
re.sub('mañana','hoy',oracion)  

'Si hoy vengo hoy voy a ver si consigo que hoy sea hoy'

Elimina palabras seguidas iguales

In [2]:
# sustituciones con ER, elimina palabras seguidas iguales, dejando solo una
oracion = 'si hooy hoy llueve llueve'
re.sub(r'(\w+) \1',r'\1',oracion)

'si hooy hoy llueve'

Elimine las líneas en blanco que puedan estar en medio de una oración

In [8]:
# Por ejemplo, sustitúyalas por un "espacio"
oracion = 'si hoy\nllueve'
print(oracion)
re.sub(r'\n',r' ',oracion)

si hoy
llueve


'si hoy llueve'

#### Carga de datos

Hay que cargar los datos para desarrollo a partir del archivo csv provisto (devel.csv). Pandas cuenta con una funcion para cargar csv. https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [9]:
df_devel = pd.read_csv('devel.csv', header=None)

Es posible ver la composición general de los datos con la función info de pandas

In [10]:
df_devel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1133 entries, 0 to 1132
Data columns (total 3 columns):
0    1133 non-null int64
1    1133 non-null object
2    1133 non-null object
dtypes: int64(1), object(2)
memory usage: 26.6+ KB


In [11]:
df_devel.head(10)

,0,1,2
0,187437263205044224,Estos días trataré de hacer deporte con mis am...,P
1,180438913880571904,Un mensaje especial para un luchador como Eric...,P
2,771051726694744064,Chicxs nunca idealicéis demasiado a una person...,N
3,810700953620021248,@wkCArlos yo soy igual Jajajaja trató de evita...,NEU
4,769894353389162497,Y yo mientras comeré palomitas mientras quedái...,N
5,817151700335267841,Lo bueno: el gato regresó. Lo malo: creo que e...,NEU
6,159520155372765184,"Recuerdo a Garzon ,bermejo,la fiscal,el poli d...",N
7,178812716499218432,Llena de tristeza que víctimas del terrorismo ...,N
8,145810899008110592,"RT @Ciutadans_Cs: A las 13h, no te pierdas a I...",NONE
9,158684073529843712,Fraga sera velado en su domicilio este lunes y...,NONE


Es posible imprimir el texto completo

In [12]:
# seleccionamos los primeros 10 tweets

for tweet in df_devel[1].values[:10]:
    print(tweet)
    

Estos días trataré de hacer deporte con mis amigos y desconectar.
Un mensaje especial para un luchador como Eric Abidal. Toda la fuerza para tu recuperación. Estamos con vos!
Chicxs nunca idealicéis demasiado a una persona, que luego os da mg en facebook y rompéis un espejo de la emoción 
@wkCArlos yo soy igual Jajajaja trató de evitarlo pero no lo logro
Y yo mientras comeré palomitas mientras quedáis retratados porque, ay, es lo que pasa cuando usas el activismo como un arma 
Lo bueno: el gato regresó. Lo malo: creo que es del vecino y no me lo puedo dejar.
Recuerdo a Garzon ,bermejo,la fiscal,el poli de la judicial cazando gratis con Correa en el calabozo...
Llena de tristeza que víctimas del terrorismo como #Pilar Manjón pretendan boicotear el trabajo de la justicia en el #11M #11Mconlasvíctimas
RT @Ciutadans_Cs: A las 13h, no te pierdas a Inés Arrimadas en la tertulia "La Alternativa de Cope Bcn #CsTeam
Fraga sera velado en su domicilio este lunes y el martes sera trasladado a Perb

In [0]:
# hacer lo mismo con el corpus de entrenamiento train los 20 primeros tweets


### Preprocesamiento

Para el procesamiento posterior del texto puede ser útil modificar el texto original para que sea más manejable. En el caso de los tweets puede tener sentido ignorar los Hashtags, Mentions y Urls para que no tenerlos en cuenta al momento de aplicar otras técnicas.

Es posible definir una función y pasarla a pandas para que la aplique en todas las filas de cierta columna. El resultado se puede guardar en una nueva columna del dataframe de pandas, por ejemplo:

In [13]:
def acentos(text):
    """
    Recibe un texto y lo devuelve sustituyendo caracteres por su versión sin acentos
    """
    return unidecode.unidecode(text)

In [14]:
df_devel['texto_limpio'] = df_devel[1].apply(acentos)

In [15]:
# proceso solo el primer tweet

for tweet in df_devel['texto_limpio'].values[:1]:
    print(tweet)

Estos dias tratare de hacer deporte con mis amigos y desconectar.


In [16]:
oracion = df_devel['texto_limpio'].values[:1]
print(oracion)

['Estos dias tratare de hacer deporte con mis amigos y desconectar.']


In [17]:
#procesar los 10 primeros

print(df_devel['texto_limpio'].values[:10])

['Estos dias tratare de hacer deporte con mis amigos y desconectar.'
 'Un mensaje especial para un luchador como Eric Abidal. Toda la fuerza para tu recuperacion. Estamos con vos!'
 'Chicxs nunca idealiceis demasiado a una persona, que luego os da mg en facebook y rompeis un espejo de la emocion '
 '@wkCArlos yo soy igual Jajajaja trato de evitarlo pero no lo logro'
 'Y yo mientras comere palomitas mientras quedais retratados porque, ay, es lo que pasa cuando usas el activismo como un arma '
 'Lo bueno: el gato regreso. Lo malo: creo que es del vecino y no me lo puedo dejar.'
 'Recuerdo a Garzon ,bermejo,la fiscal,el poli de la judicial cazando gratis con Correa en el calabozo...'
 'Llena de tristeza que victimas del terrorismo como #Pilar Manjon pretendan boicotear el trabajo de la justicia en el #11M #11Mconlasvictimas'
 'RT @Ciutadans_Cs: A las 13h, no te pierdas a Ines Arrimadas en la tertulia "La Alternativa de Cope Bcn #CsTeam'
 'Fraga sera velado en su domicilio este lunes y el 

En el caso de sustituir expresiones pueden usarse expresiones regulares:

In [18]:
import re
def hashtags(text):
    """
    Sustituye los hashtags con la palabra HASHTAG
    """
    return re.sub('\s([#][\w_-]+)', ' HASHTAG ', text)

Se aplica la función sobre la columna texto_limpio de manera de acumular las transformaciones

In [84]:
df_devel['texto_limpio'] = df_devel[1].apply(hashtags)

In [85]:
# Aplicarlo a los 10 primeros tweets
print(df_devel['texto_limpio'].values[:10])

['Estos días trataré de hacer deporte con mis amigos y desconectar.'
 'Un mensaje especial para un luchador como Eric Abidal. Toda la fuerza para tu recuperación. Estamos con vos!'
 'Chicxs nunca idealicéis demasiado a una persona, que luego os da mg en facebook y rompéis un espejo de la emoción '
 '@wkCArlos yo soy igual Jajajaja trató de evitarlo pero no lo logro'
 'Y yo mientras comeré palomitas mientras quedáis retratados porque, ay, es lo que pasa cuando usas el activismo como un arma '
 'Lo bueno: el gato regresó. Lo malo: creo que es del vecino y no me lo puedo dejar.'
 'Recuerdo a Garzon ,bermejo,la fiscal,el poli de la judicial cazando gratis con Correa en el calabozo...'
 'Llena de tristeza que víctimas del terrorismo como HASHTAG  Manjón pretendan boicotear el trabajo de la justicia en el HASHTAG  HASHTAG '
 'RT @Ciutadans_Cs: A las 13h, no te pierdas a Inés Arrimadas en la tertulia "La Alternativa de Cope Bcn HASHTAG '
 'Fraga sera velado en su domicilio este lunes y el mar

Hay que hacer lo propio con las mentions (@), con las URL, simbolos de puntuación y las cifras.

In [86]:
def mentions(text):
    """
    
    """
    return re.sub('(^@| @)(\w|\d|_)+ ','MENTION ',text)

In [87]:
def url(text):
    """
    
    """
    return re.sub(' ((https?://|www\.)\S+|\S+\.com)( |\.)',' URL ',text)

In [88]:
def numeros(text):
    """
    
    """
    return re.sub(' \d+(\.\d+)?\.? ',' NUMBER ',text)

In [89]:
def puntuacion(text):
    """
    
    """
    return re.sub('[.,!?:;"]',' PUNCT',text)

Pueden aplicarse las transformaciones de forma encadenada:

In [90]:
df_devel['texto_limpio'] = df_devel[1].apply(acentos).apply(hashtags).apply(mentions).apply(url).apply(numeros).apply(puntuacion)

In [91]:
# verificarlo con los 10 primeros tweets
print(df_devel['texto_limpio'].values[:10])

['Estos dias tratare de hacer deporte con mis amigos y desconectar PUNCT'
 'Un mensaje especial para un luchador como Eric Abidal PUNCT Toda la fuerza para tu recuperacion PUNCT Estamos con vos PUNCT'
 'Chicxs nunca idealiceis demasiado a una persona PUNCT que luego os da mg en facebook y rompeis un espejo de la emocion '
 'MENTION yo soy igual Jajajaja trato de evitarlo pero no lo logro'
 'Y yo mientras comere palomitas mientras quedais retratados porque PUNCT ay PUNCT es lo que pasa cuando usas el activismo como un arma '
 'Lo bueno PUNCT el gato regreso PUNCT Lo malo PUNCT creo que es del vecino y no me lo puedo dejar PUNCT'
 'Recuerdo a Garzon  PUNCTbermejo PUNCTla fiscal PUNCTel poli de la judicial cazando gratis con Correa en el calabozo PUNCT PUNCT PUNCT'
 'Llena de tristeza que victimas del terrorismo como HASHTAG  Manjon pretendan boicotear el trabajo de la justicia en el HASHTAG  HASHTAG '
 'RT @Ciutadans_Cs PUNCT A las 13h PUNCT no te pierdas a Ines Arrimadas en la tertulia 

Hay que tener cuidado por si exiten tweets con líneas en blanco
¿Cómo lo resolvería?

In [27]:
df_devel['texto_limpio'] = df_devel['texto_limpio'].apply(lambda text : re.sub(r'\n\s*4\n','\n',text))

### Trabajo con el tokenizador de NLTK, el punkt

In [28]:
import nltk

# es importante para tokenizar (palabras u oraciones) bajar el paquete punkt

nltk.download('punkt')

[nltk_data] Downloading package punkt to /Users/Bruno/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [29]:
# como invocar al tokenizador de palabras

nltk.word_tokenize('Hoy es un lindo dia')

['Hoy', 'es', 'un', 'lindo', 'dia']

In [30]:
# hacerlo para el primer tweet
nltk.word_tokenize(df_devel['texto_limpio'][0])

['Estos',
 'dias',
 'tratare',
 'de',
 'hacer',
 'deporte',
 'con',
 'mis',
 'amigos',
 'y',
 'desconectar',
 'PUNCT']

In [31]:
# como invocar al tokenizador de oraciones

nltk.sent_tokenize('Hola amigos. Gracias por venir. Saludos. Qué lindoo!! Será verdad? Tal vez si')

['Hola amigos.',
 'Gracias por venir.',
 'Saludos.',
 'Qué lindoo!!',
 'Será verdad?',
 'Tal vez si']

### Spacy

Tokenizamos con Spacy

In [41]:
# con la bilbioteca spacy también se puede tokenizar
# !python -m spacy download es_core_news_md
!python -m spacy download es_core_web_md

import spacy

nlp = spacy.load('es_core_web_md')

doc = nlp("Hoy puede ser un gran día.")
for token in doc:
    print(token.text)



    100% |████████████████████████████████| 395.1MB 799kB/s 
  Running setup.py install for es-core-web-md ... - \ | / - \ | / - \ done
You are using pip version 9.0.1, however version 19.3.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.

    Linking successful

    /Users/Bruno/anaconda/lib/python3.5/site-packages/es_core_web_md/es_core_web_md-1.0.0
    -->
    /Users/Bruno/anaconda/lib/python3.5/site-packages/spacy/data/es_core_web_md

    You can now load the model via spacy.load('es_core_web_md').

Hoy
puede
ser
un
gran
día
.


Spacy aplica un pipeline de PLN a los textos, tokenizador, stemming, pos tagger, ner, vectores de palabras.

In [42]:
docs = []
for tweet in df_devel['texto_limpio'].values[:10]:
    doc = nlp(tweet)
    docs.append(doc)

In [43]:
for token in doc:
    print(token.text, token.pos_, token.dep_)

Fraga PROPN ROOT
sera AUX amod
velado VERB ROOT
en ADP case
su DET det
domicilio NOUN obl
este DET det
lunes NOUN appos
y CONJ cc
el DET det
martes NOUN conj
sera NOUN obj
trasladado ADJ amod
a ADP case
Perbes(Lugo PROPN obj
) PUNCT punct
donde PRON obl
sera AUX aux
enterrado VERB acl
PUNCT PROPN nsubj
  SPACE 
Es AUX cop
la DET det
intencion NOUN advcl
de ADP case
la DET det
familia NOUN nmod
PUNCT PROPN appos


In [44]:
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)

Fraga fraga PROPN PROPN___ ROOT Xxxxx True False
sera sera AUX AUX__Mood=Ind|Number=Sing|Person=3|Tense=Imp|VerbForm=Fin amod xxxx True True
velado velado VERB VERB__Gender=Masc|Number=Sing|Tense=Past|VerbForm=Part ROOT xxxx True False
en en ADP ADP__AdpType=Prep case xx True True
su su DET DET__Number=Sing|Person=3|Poss=Yes|PronType=Prs det xx True True
domicilio domicilio NOUN NOUN__Gender=Masc|Number=Sing obl xxxx True False
este este DET DET__Gender=Masc|Number=Sing|PronType=Dem det xxxx True True
lunes lunes NOUN NOUN__AdvType=Tim appos xxxx True False
y y CONJ CCONJ___ cc x True False
el el DET DET__Definite=Def|Gender=Masc|Number=Sing|PronType=Art det xx True True
martes martes NOUN NOUN__AdvType=Tim conj xxxx True False
sera sera NOUN NOUN___ obj xxxx True True
trasladado trasladado ADJ ADJ__Gender=Masc|Number=Sing|VerbForm=Part amod xxxx True False
a a ADP ADP__AdpType=Prep case x True False
Perbes(Lugo perbes(lugo PROPN PROPN___ obj Xxxxx(Xxxx False False
) ) PUNCT PUNCT__Pun

In [45]:
tokens = nlp("Perros y gatos se pelean por los peces")

In [46]:
# Permite verificar con los vectores de palabras cuan cerca están 2 palabras utilizando
# el corpus de noticias de la wikipedia

for token1 in tokens:
    for token2 in tokens:
        print(token1.text, token2.text, token1.similarity(token2))

Perros Perros -3.36589109172e-21
Perros y 0.0401328898052
Perros gatos 3.76877257979e-21
Perros se 0.0
Perros pelean -0.0959286250882
Perros por -0.0418655873622
Perros los 0.0388161976578
Perros peces 0.0
y Perros 0.0401328898052
y y -1.4062461959e-21
y gatos 0.0449366098082
y se 0.0
y pelean 0.0
y por -1.4669594752e-21
y los -4.6282132749e+17
y peces -1.8750773402e-21
gatos Perros 3.76877257979e-21
gatos y 0.0449366098082
gatos gatos 0.0
gatos se -2.32256748419e-21
gatos pelean 0.0
gatos por -8.64722546825e+17
gatos los -0.0434623157429
gatos peces 0.0
se Perros 0.0
se y -1.34075405865e-21
se gatos -0.0428438079748
se se 4.34987047207e+17
se pelean -0.0591175775974
se por 0.0258003500756
se los 0.0
se peces 0.0
pelean Perros -0.0959286250882
pelean y 0.0
pelean gatos 0.0
pelean se -0.0591175775974
pelean pelean 8.03446448215e-21
pelean por 0.0
pelean los 0.0
pelean peces -1.52512843332e+18
por Perros -0.0418655873622
por y -1.4669594752e-21
por gatos -8.64722546825e+17
por se 0.02580

Hacerlo con otra oración y analizar el resultado


### Análisis

Interesa conocer la longitud de los textos, se puede crear una nueva columna con ese valor usando la funcion len

In [92]:
df_devel['largo_texto'] = df_devel['texto_limpio'].apply(len)
# df_devel['largo_texto'] = df_devel[1].apply(len)

In [93]:
df_devel.head(5)

,0,1,2,texto_limpio,largo_texto
0,187437263205044224,Estos días trataré de hacer deporte con mis am...,P,Estos dias tratare de hacer deporte con mis am...,70
1,180438913880571904,Un mensaje especial para un luchador como Eric...,P,Un mensaje especial para un luchador como Eric...,123
2,771051726694744064,Chicxs nunca idealicéis demasiado a una person...,N,Chicxs nunca idealiceis demasiado a una person...,119
3,810700953620021248,@wkCArlos yo soy igual Jajajaja trató de evita...,NEU,MENTION yo soy igual Jajajaja trato de evitarl...,64
4,769894353389162497,Y yo mientras comeré palomitas mientras quedái...,N,Y yo mientras comere palomitas mientras quedai...,134


La función describe de pandas brinda estadisticas sobre las columnas númericas, lo que permite saber por ejemplo que el largo primedio de los textos es 98 carácteres y el máximo es 201.

In [94]:
df_devel.describe()

,0,largo_texto
count,1.133000e+03,1133.000000
mean,3.919435e+17,107.505737
std,3.006609e+17,42.293932
min,1.424945e+17,11.000000
25%,1.624782e+17,73.000000
50%,1.795517e+17,113.000000
75%,7.706669e+17,141.000000
max,8.194038e+17,403.000000


Es posible localizar la fila donde se encuentra el tweet más largo:

In [95]:
# df_devel.loc[df_devel['largo_texto'].idxmax()]['texto_limpio']
df_devel.loc[df_devel['largo_texto'].idxmax()]['texto_limpio']

'Todo estaba bien hasta que me desilusionaron  PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCTme siento herida  PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT PUNCTque dolor  PUNCT PUNCT PUNCT PUNCT PUNCT PUNCT  PUNCT PUNCT PUNCT PUNCT PUNCT'

In [96]:
df_devel[1][505]

'Todo estaba bien hasta que me desilusionaron .....................me siento herida .......................que dolor ...... .....'

In [97]:
df_devel['texto_limpio'][15]

'Que chiva pasar debajo del puente de la platina PUNCT siempre he querido  PUNCT PUNCT PUNCTes en serio PUNCT'

In [82]:
df_devel['largo_texto'].idxmax()

505